In [ ]:
import json 
import matplotlib
from matplotlib import pyplot as plt
import pickle
import pandas as pd
import numpy as np 
import sklearn 
import statistics
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
%matplotlib inline


In [ ]:
full_data = pd.read_csv('../../data/raw-data/filtered_everything.csv', index_col=0)
combined_test = pd.read_csv('../../data/splitted-data/everything/nbins-10/test_set.csv', index_col=0)
m2575 = pd.read_csv('../../data/raw-data/unprocessed/mixed-25-75.csv', index_col=0)
m2575_test = pd.read_csv('../../data/splitted-data/mixed2575/nbins-10/test_set.csv', index_col=0)
m5050_test = pd.read_csv('../../data/splitted-data/mixed5050/nbins-10/test_set.csv', index_col=0)
optimal = pd.read_csv('../../data/raw-data/opt_22_raw.csv', index_col=0)


In [ ]:
oresults_path = '../predicted-results/original/nbins-10'
mresults_path = '../predicted-results/mixed5050/nbins-10'
eresults_path = '../predicted-results/everything/nbins-10'

omodels_path = '../models/original'
mmodels_path = '../models/mixed5050/nbins-10'
emodels_path = '../models/everything/nbins-10'

test_sets = ['5050', '2575', 'everything']

mpoints = [100, 200, 300, 500, 1000, 1500, 2000, 2500, 'all']
epoints = [100, 200, 300, 500, 1000, 1500, 2000, 2500, 4000, 6000, 'all']


In [ ]:
# Consider 5050 models and everything models at the 2500 mark 
m5models = dict()
emodels = dict()
omodels = dict()
# 5050 model
for tset in ['5050', '2575', 'everything']:
    m5models[tset] = dict()
    emodels[tset] = dict()
    omodels[tset] = dict()
    
    '''First load the original models'''
    for target in ['COF', 'intercept']:
        with open(f'{omodels_path}/{target}.pickle', 'rb') as f:
            model = pickle.load(f)
        with open(f'{omodels_path}/{target}.ptxt', 'rb') as f:
            features = pickle.load(f)
        with open(f'{oresults_path}/{target}_on_{tset}.json', 'r') as f:
            data = json.load(f)
        omodels[tset][target] = {
#                                 'model': model,
                                 'features': features,
                                 'data': data,
                                 'n_train': len(model.oob_prediction_),
                                 'r_square': data[target]['r_square']}
        
    '''Then load the mixed5050 models'''
    for target in ['COF', 'intercept']:
        m5models[tset][target] = dict()
        for point in mpoints:
            m5models[tset][target][point] = dict()
            for i in range(5):
                with open(f'{mmodels_path}/set_{i}/{target}_{point}.pickle', 'rb') as f:
                    model = pickle.load(f)
                with open(f'{mmodels_path}/set_{i}/{target}_{point}.ptxt', 'rb') as f:
                    features = pickle.load(f)
                with open(f'{mresults_path}/set_{i}/{target}_{point}_on_{tset}.json', 'r') as f :
                    data = json.load(f)
                
                m5models[tset][target][point][f'set_{i}'] = dict()
                m5models[tset][target][point][f'set_{i}'] = {
#                  'model': model,
                  'features': features,
                  'data': data,
                  'n_train': len(model.oob_prediction_),
                  'r_square': data[target]['r_square']}

                    
    '''Finally load the combined models'''
    for target in ['COF', 'intercept']:
        emodels[tset][target] = dict()
        for point in epoints:
            emodels[tset][target][point] = dict()
            for i in range(5):
                with open(f'{emodels_path}/set_{i}/{target}_{point}.pickle', 'rb') as f:
                    model = pickle.load(f)
                with open(f'{emodels_path}/set_{i}/{target}_{point}.ptxt', 'rb') as f:
                    features = pickle.load(f)
                with open(f'{eresults_path}/set_{i}/{target}_{point}_on_{tset}.json', 'r') as f :
                    data = json.load(f)
                    
                emodels[tset][target][point][f'set_{i}'] = dict()
                emodels[tset][target][point][f'set_{i}'] = {
#                  'model': model,
                  'features': features,
                  'data': data,
                  'n_train': len(model.oob_prediction_),
                  'r_square': data[target]['r_square']}
 


In [ ]:
doi = {'omodels': dict(), 
       'mmodels': dict(), 
       'emodels': dict()}

for tset in ['5050', '2575', 'everything']:
    doi['omodels'][f'on_{tset}'] = dict()
    for target in ['COF', 'intercept']:
        doi['omodels'][f'on_{tset}'][target] = dict()
        omoi = omodels[tset][target]
        odoi = dict() 
        odoi['terminal_group_1'] = list()
        odoi['terminal_group_2'] = list()
        odoi['terminal_group_3'] = list()
        odoi['frac-1'] = list() 
        odoi['frac-2'] = list()
        odoi[f'predicted-{target}'] = list()
        odoi[f'simulated-{target}'] = list()
        for j in omoi['data'][target]:
            if j != 'r_square':
                odoi['terminal_group_1'].append(omoi['data'][target][j]['tg-1'])
                odoi['terminal_group_2'].append(omoi['data'][target][j]['tg-2'])
                odoi['terminal_group_3'].append(omoi['data'][target][j]['tg-3'])
                odoi['frac-1'].append(omoi['data'][target][j]['frac-1'])
                odoi['frac-2'].append(omoi['data'][target][j]['frac-2'])
                odoi[f'predicted-{target}'].append(omoi['data'][target][j][f'predicted-{target}'])
                odoi[f'simulated-{target}'].append(omoi['data'][target][j][f'simulated-{target}'])

        doi['omodels'][f'on_{tset}'][target] = pd.DataFrame.from_dict(odoi)            

    for point in mpoints: 
        doi['mmodels'][f'{point}_on_{tset}'] = dict() 
        for target in ['COF', 'intercept']:
            doi['mmodels'][f'{point}_on_{tset}'][target] = dict()
            for i in range(5):
                doi['mmodels'][f'{point}_on_{tset}'][target][f'set_{i}'] = dict()
                mmoi = m5models[tset][target][point][f'set_{i}']
                mdoi = dict() 
                mdoi['terminal_group_1'] = list()
                mdoi['terminal_group_2'] = list()
                mdoi['terminal_group_3'] = list()
                mdoi['frac-1'] = list() 
                mdoi['frac-2'] = list()
                mdoi[f'predicted-{target}'] = list()
                mdoi[f'simulated-{target}'] = list()
                for j in mmoi['data'][target]:
                    if j != 'r_square':
                        mdoi['terminal_group_1'].append(mmoi['data'][target][j]['tg-1'])
                        mdoi['terminal_group_2'].append(mmoi['data'][target][j]['tg-2'])
                        mdoi['terminal_group_3'].append(mmoi['data'][target][j]['tg-3'])
                        mdoi['frac-1'].append(mmoi['data'][target][j]['frac-1'])
                        mdoi['frac-2'].append(mmoi['data'][target][j]['frac-2'])
                        mdoi[f'predicted-{target}'].append(mmoi['data'][target][j][f'predicted-{target}'])
                        mdoi[f'simulated-{target}'].append(mmoi['data'][target][j][f'simulated-{target}'])

                doi['mmodels'][f'{point}_on_{tset}'][target][f'set_{i}'] = pd.DataFrame.from_dict(mdoi)
                           
    for point in epoints: 
        doi['emodels'][f'{point}_on_{tset}'] = dict() 
        for target in ['COF', 'intercept']:
            doi['emodels'][f'{point}_on_{tset}'][target] = dict()
            for i in range(5):
                doi['emodels'][f'{point}_on_{tset}'][target][f'set_{i}'] = dict()
                emoi = emodels[tset][target][point][f'set_{i}']
                edoi = dict() 
                edoi['terminal_group_1'] = list()
                edoi['terminal_group_2'] = list()
                edoi['terminal_group_3'] = list()
                edoi['frac-1'] = list() 
                edoi['frac-2'] = list()
                edoi[f'predicted-{target}'] = list()
                edoi[f'simulated-{target}'] = list()
                for j in emoi['data'][target]:
                    if j != 'r_square':
                        edoi['terminal_group_1'].append(emoi['data'][target][j]['tg-1'])
                        edoi['terminal_group_2'].append(emoi['data'][target][j]['tg-2'])
                        edoi['terminal_group_3'].append(emoi['data'][target][j]['tg-3'])
                        edoi['frac-1'].append(emoi['data'][target][j]['frac-1'])
                        edoi['frac-2'].append(emoi['data'][target][j]['frac-2'])
                        edoi[f'predicted-{target}'].append(emoi['data'][target][j][f'predicted-{target}'])
                        edoi[f'simulated-{target}'].append(emoi['data'][target][j][f'simulated-{target}'])

                doi['emodels'][f'{point}_on_{tset}'][target][f'set_{i}'] = pd.DataFrame.from_dict(edoi)
                           
                

In [ ]:
"""Code to rank top and bottom by %"""

refs = {'5050' : m5050_test,
        '2575' : m2575_test, 
        'everything' : combined_test} 

identifiers = ['terminal_group_1', 'terminal_group_2', 'terminal_group_3', 'frac-1', 'frac-2']
k = 300
ps = [0.1, 0.15, 0.2 , 0.3, 0.4, 0.5]
plot_data = dict()
for tset in ['5050', '2575', 'everything']:
    for p in ps:
        plot_data[f'top_{p}_{tset}'] = dict()
        plot_data[f'bot_{p}_{tset}'] = dict()
        
        plot_data[f'top_{p}_{tset}']['og'] ={'COF': list(),
                                             'intercept': list(),
                                             'opt': list(),
                                             'point': list()}
        plot_data[f'bot_{p}_{tset}']['og'] ={'COF': list(), 
                                             'intercept': list(), 
                                             'point': list()}
        plot_data[f'top_{p}_{tset}']['m50'] ={'COF': list(),
                                              'intercept': list(),
                                              'opt': list(),
                                              'point': list()}
        plot_data[f'bot_{p}_{tset}']['m50'] ={'COF': list(), 
                                              'intercept': list(),
                                              'point': list()}
        plot_data[f'top_{p}_{tset}']['eve'] ={'COF': list(),
                                              'intercept': list(),
                                              'opt': list(),
                                              'point': list()}
        plot_data[f'bot_{p}_{tset}']['eve'] ={'COF': list(), 
                                              'intercept': list(),
                                              'point': list()}
        
        sorted_preds = dict()
        sorted_refs = dict()
        for target in ['COF', 'intercept']:
            sorted_preds[target] = doi['omodels'][f'on_{tset}'][target].sort_values(by=f'predicted-{target}')
            sorted_refs[target] = refs[tset].sort_values(by=target)
            n = round(len(refs[tset])*p)
            top_intersect = pd.merge(sorted_refs[target].head(n), sorted_preds[target].head(n), how='inner',
                                     on=identifiers)
            bot_intersect = pd.merge(sorted_refs[target].tail(n), sorted_preds[target].tail(n), how='inner',
                                     on=identifiers)
            plot_data[f'top_{p}_{tset}']['og'][target].append(len(top_intersect)/n*100)
            plot_data[f'bot_{p}_{tset}']['og'][target].append(len(bot_intersect)/n*100)
                
            
        preds_opt = pd.merge(sorted_preds['COF'].head(k), sorted_preds['intercept'].head(k), how='inner',
                                 on=identifiers)
        opt_intersect = pd.merge(preds_opt, optimal, how='inner', on=identifiers)
        
        plot_data[f'top_{p}_{tset}']['og']['opt'].append(len(opt_intersect)/len(optimal)*100) 
        plot_data[f'top_{p}_{tset}']['og']['point'].append(100) 
        plot_data[f'bot_{p}_{tset}']['og']['point'].append(100)

        
        for point in mpoints:
            if point == 'all':
                plot_data[f'top_{p}_{tset}']['m50']['point'].append(2680) 
                plot_data[f'bot_{p}_{tset}']['m50']['point'].append(2680)
            else: 
                plot_data[f'top_{p}_{tset}']['m50']['point'].append(point) 
                plot_data[f'bot_{p}_{tset}']['m50']['point'].append(point)
                
            tmp = {'top': {'COF': list(), 'intercept': list(), 'opt': list()},
                   'bot': {'COF': list(), 'intercept': list()}}
            for i in range(5):
                sorted_preds = dict()
                sorted_refs = dict()
                for target in ['COF', 'intercept']:
                    sorted_preds[target] = doi['mmodels'][f'{point}_on_{tset}'][target][f'set_{i}'].sort_values(by=f'predicted-{target}')
                    sorted_refs[target] = refs[tset].sort_values(by=target)
                    n = round(len(refs[tset])*p)
                    top_intersect = pd.merge(sorted_refs[target].head(n), sorted_preds[target].head(n), how='inner',
                                             on=identifiers)
                    bot_intersect = pd.merge(sorted_refs[target].tail(n), sorted_preds[target].tail(n), how='inner',
                                             on=identifiers)

#                     plot_data[f'top_{p}_{tset}']['m50'][target].append(len(top_intersect)/n*100)
#                     plot_data[f'bot_{p}_{tset}']['m50'][target].append(len(bot_intersect)/n*100)
                    tmp['top'][target].append(len(top_intersect)/n*100)
                    tmp['bot'][target].append(len(bot_intersect)/n*100)

                preds_opt = pd.merge(sorted_preds['COF'].head(k), sorted_preds['intercept'].head(k), how='inner',
                                     on=identifiers)
                opt_intersect = pd.merge(preds_opt, optimal, how='inner', on=identifiers)
                tmp['top']['opt'].append(len(opt_intersect)/len(optimal)*100) 
                
            for target in ['COF', 'intercept']:
                for end in ['top', 'bot']:
                    plot_data[f'{end}_{p}_{tset}']['m50'][target].append(tmp[end][target])
            plot_data[f'top_{p}_{tset}']['m50']['opt'].append(tmp['top']['opt'])


        for point in epoints:
            if point == 'all':
                plot_data[f'top_{p}_{tset}']['eve']['point'].append(7816)
                plot_data[f'bot_{p}_{tset}']['eve']['point'].append(7816)
            else:
                plot_data[f'top_{p}_{tset}']['eve']['point'].append(point)
                plot_data[f'bot_{p}_{tset}']['eve']['point'].append(point)
                                                                    
            tmp = {'top': {'COF': list(), 'intercept': list(), 'opt': list()},
                   'bot': {'COF': list(), 'intercept': list()}}
            for i in range(5):
                sorted_preds = dict()
                sorted_refs = dict()
                for target in ['COF', 'intercept']:
                    sorted_preds[target] = doi['emodels'][f'{point}_on_{tset}'][target][f'set_{i}'].sort_values(by=f'predicted-{target}')
                    sorted_refs[target] = refs[tset].sort_values(by=target)
                    n = round(len(refs[tset])*p)
                    top_intersect = pd.merge(sorted_preds[target].head(n), sorted_refs[target].head(n), how='inner',
                                             on=identifiers)
                    bot_intersect = pd.merge(sorted_preds[target].tail(n), sorted_refs[target].tail(n), how='inner',
                                             on=identifiers)

                    tmp['top'][target].append(len(top_intersect)/n*100)
                    tmp['bot'][target].append(len(bot_intersect)/n*100)


                preds_opt = pd.merge(sorted_preds['COF'].head(k), sorted_preds['intercept'].head(k), how='inner',
                                         on=identifiers)
                opt_intersect = pd.merge(preds_opt, optimal, how='inner', on=identifiers)
                tmp['top']['opt'].append(len(opt_intersect)/len(optimal)*100) 
            
            for target in ['COF', 'intercept']:
                for end in ['top', 'bot']:
                    plot_data[f'{end}_{p}_{tset}']['eve'][target].append(tmp[end][target])
                    
            plot_data[f'top_{p}_{tset}']['eve']['opt'].append(tmp['top']['opt'])


# Top 20% and 15% of 5050-test and 2575-test predicted by m50 models

In [ ]:
markersize=10
elinewidth=1
alpha=0.5

In [ ]:

n = 0.2

fig1, ax = plt.subplots(2, 2, figsize=(20, 14))

markers = {'COF': ['o', '--', 'dodgerblue'], 'intercept': ['s', '--', 'firebrick'], 'opt': ['s', '--', 'indigo']}

for target in ['COF', 'intercept']:
    ax[0, 0].errorbar(x=plot_data[f'top_{n}_5050']['m50']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'top_{n}_5050']['m50'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'top_{n}_5050']['m50'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)
    ax[1, 0].errorbar(x=plot_data[f'top_{n}_2575']['m50']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'top_{n}_2575']['m50'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'top_{n}_2575']['m50'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)

    ax[0, 0].axhline(plot_data[f'top_{n}_5050']['og'][target],
                linestyle=markers[target][1],
                linewidth=2,
                color=markers[target][2],
                alpha=alpha)
    ax[1, 0].axhline(plot_data[f'top_{n}_2575']['og'][target],
                linestyle=markers[target][1],
                linewidth=2,
                color=markers[target][2],
                alpha=alpha)

    ax[0, 1].errorbar(x=plot_data[f'bot_{n}_5050']['m50']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'bot_{n}_5050']['m50'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'bot_{n}_5050']['m50'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)
    ax[1, 1].errorbar(x=plot_data[f'bot_{n}_2575']['m50']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'bot_{n}_2575']['m50'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'bot_{n}_2575']['m50'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)

    ax[0, 1].axhline(plot_data[f'bot_{n}_5050']['og'][target],
                linestyle=markers[target][1],
                linewidth=2,
                color=markers[target][2],
                alpha=alpha)
    ax[1, 1].axhline(plot_data[f'bot_{n}_2575']['og'][target],
                linestyle=markers[target][1],
                linewidth=2,
                color=markers[target][2],
                alpha=alpha)
    
#ax.legend()
'''
ax[0, 0].set_ylim(50, 100)
ax[0, 0].set_xlim(0, 3000)
ax[1, 0].set_ylim(50, 100)
ax[1, 0].set_xlim(0, 3000)

ax[0, 0].set_xlabel('N', style='italic')
ax[0, 0].set_ylabel('Overlapped (%)')
ax[1, 0].set_xlabel('N', style='italic')
ax[1, 0].set_ylabel('Overlapped (%)')


ax[0, 1].set_ylim(50, 100)
ax[0, 1].set_xlim(0, 3000)
ax[1, 1].set_ylim(50, 100)
ax[1, 1].set_xlim(0, 3000)

ax[0, 1].set_xlabel('N', style='italic')
ax[0, 1].set_ylabel('Overlapped (%)')
ax[1, 1].set_xlabel('N', style='italic')
ax[1, 1].set_ylabel('Overlapped (%)')
'''
leg1_ele = [Line2D([0], [0], color='dodgerblue', marker='o', markersize=12, linestyle='', label='COF'),
                   Line2D([0], [0], color='firebrick', marker='s', markersize=12, linestyle='', label=r'$\mathit{F_0}$')]

for i in [0, 1]:
    for j in [0, 1]:
        legend = ax[i, j].legend(handles=leg1_ele, loc=4)# title='Properties')
        ax[i, j].set_ylim(45, 100)
        ax[i, j].set_xlim(0, 3000)
        ax[i, j].set_xlabel('N', style='italic')
        ax[i, j].set_ylabel('Overlapped (%)')
        ax[i, j].add_artist(legend)

#plt.savefig(fname=f'./plots/top_and_bot/overlap_{n}_tm_m50_v2.pdf', dpi=500, bbox_inches='tight')


plt.show()
plt.close()

In [ ]:

n = 0.15

fig1, ax = plt.subplots(2, 2, figsize=(20, 14))

markers = {'COF': ['o', '--', 'dodgerblue'], 'intercept': ['s', '--', 'firebrick'], 'opt': ['s', '--', 'indigo']}

for target in ['COF', 'intercept']:
    ax[0, 0].errorbar(x=plot_data[f'top_{n}_5050']['m50']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'top_{n}_5050']['m50'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'top_{n}_5050']['m50'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)
    ax[1, 0].errorbar(x=plot_data[f'top_{n}_2575']['m50']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'top_{n}_2575']['m50'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'top_{n}_2575']['m50'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)

    ax[0, 0].axhline(plot_data[f'top_{n}_5050']['og'][target],
                linestyle=markers[target][1],
                linewidth=2,
                color=markers[target][2],
                alpha=alpha)
    ax[1, 0].axhline(plot_data[f'top_{n}_2575']['og'][target],
                linestyle=markers[target][1],
                linewidth=2,
                color=markers[target][2],
                alpha=alpha)

    ax[0, 1].errorbar(x=plot_data[f'bot_{n}_5050']['m50']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'bot_{n}_5050']['m50'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'bot_{n}_5050']['m50'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)
    ax[1, 1].errorbar(x=plot_data[f'bot_{n}_2575']['m50']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'bot_{n}_2575']['m50'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'bot_{n}_2575']['m50'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)

    ax[0, 1].axhline(plot_data[f'bot_{n}_5050']['og'][target],
                linestyle=markers[target][1],
                linewidth=2,
                color=markers[target][2],
                alpha=alpha)
    ax[1, 1].axhline(plot_data[f'bot_{n}_2575']['og'][target],
                linestyle=markers[target][1],
                linewidth=2,
                color=markers[target][2],
                alpha=alpha)
    
#ax.legend()
'''
ax[0, 0].set_ylim(50, 100)
ax[0, 0].set_xlim(0, 3000)
ax[1, 0].set_ylim(50, 100)
ax[1, 0].set_xlim(0, 3000)

ax[0, 0].set_xlabel('N', style='italic')
ax[0, 0].set_ylabel('Overlapped (%)')
ax[1, 0].set_xlabel('N', style='italic')
ax[1, 0].set_ylabel('Overlapped (%)')


ax[0, 1].set_ylim(50, 100)
ax[0, 1].set_xlim(0, 3000)
ax[1, 1].set_ylim(50, 100)
ax[1, 1].set_xlim(0, 3000)

ax[0, 1].set_xlabel('N', style='italic')
ax[0, 1].set_ylabel('Overlapped (%)')
ax[1, 1].set_xlabel('N', style='italic')
ax[1, 1].set_ylabel('Overlapped (%)')
'''
leg1_ele = [Line2D([0], [0], color='dodgerblue', marker='o', markersize=12, linestyle='', label='COF'),
                   Line2D([0], [0], color='firebrick', marker='s', markersize=12, linestyle='', label=r'$\mathit{F_0}$')]

for i in [0, 1]:
    for j in [0, 1]:
        legend = ax[i, j].legend(handles=leg1_ele, loc=4)# title='Properties')
        ax[i, j].set_ylim(40, 100)
        ax[i, j].set_xlim(0, 3000)
        ax[i, j].set_xlabel('N', style='italic')
        ax[i, j].set_ylabel('Overlapped (%)')
        ax[i, j].add_artist(legend)

#plt.savefig(fname=f'./plots/top_and_bot/overlap_{n}_tm_m50_v2.pdf', dpi=500, bbox_inches='tight')


plt.show()
plt.close()

# Top 20% and 15% of 5050-test predicted by m50 amd eve models

In [ ]:
# Figure 1: Top 20% of t5050 with predicted by m50 and eve models 

n = 0.2

fig1, ax = plt.subplots(1, 2, figsize=(20, 7))

markers = {'COF': ['o', '--', 'dodgerblue'], 'intercept': ['s', '--', 'firebrick'], 'opt': ['P', '--', 'indigo']}

for target in ['COF', 'intercept', 'opt']:
    ax[0].errorbar(x=plot_data[f'top_{n}_everything']['eve']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'top_{n}_everything']['eve'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'top_{n}_everything']['eve'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)
    ax[0].axhline(plot_data[f'top_{n}_everything']['og'][target],
                linestyle=markers[target][1],
                linewidth=2,
                color=markers[target][2],
                alpha=alpha)

    if target != 'opt':
        ax[1].errorbar(x=plot_data[f'bot_{n}_everything']['eve']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'top_{n}_everything']['eve'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'top_{n}_everything']['eve'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)

        ax[1].axhline(plot_data[f'top_{n}_everything']['og'][target],
                    linestyle=markers[target][1],
                    linewidth=2,
                    color=markers[target][2],
                    alpha=alpha)

ax[0].set_xlabel('N', style='italic')
ax[0].set_ylabel('Overlapped (%)')
ax[1].set_xlabel('N', style='italic')
ax[1].set_ylabel('Overlapped (%)')

ax[0].set_ylim(0, 100)
ax[1].set_ylim(0, 100)
ax[0].set_xlim(0, 8000)
ax[1].set_xlim(0, 8000)

leg1_ele = [Line2D([0], [0], color='dodgerblue', marker='o', markersize=12, linestyle='', label='COF'),
            Line2D([0], [0], color='firebrick', marker='s', markersize=12, linestyle='', label=r'$\mathit{F_0}$'),
            Line2D([0], [0], color='indigo', marker='P', markersize=12, linestyle='', label='Optimal')]
leg2_ele = [Line2D([0], [0], color='dodgerblue', marker='o', markersize=12, linestyle='', label='COF'),
            Line2D([0], [0], color='firebrick', marker='s', markersize=12, linestyle='', label=r'$\mathit{F_0}$')]                         

legend1a = ax[0].legend(handles=leg1_ele, loc=4)#, title='Properties')
ax[0].add_artist(legend1a)

legend1b = ax[1].legend(handles=leg2_ele, loc=4)#, title='Properties')
ax[1].add_artist(legend1b)

#plt.savefig(fname=f'./plots/top_and_bot/overlap_{n}_teve_eve.pdf', dpi=500, bbox_inches='tight')


plt.show()

In [ ]:
# Figure 1: Top 20% of t5050 with predicted by m50 and eve models 

n = 0.15

fig1, ax = plt.subplots(1, 2, figsize=(20, 7))

markers = {'COF': ['o', '--', 'dodgerblue'], 'intercept': ['s', '--', 'firebrick'], 'opt': ['P', '--', 'indigo']}

for target in ['COF', 'intercept', 'opt']:
    ax[0].errorbar(x=plot_data[f'top_{n}_everything']['eve']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'top_{n}_everything']['eve'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'top_{n}_everything']['eve'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)
    ax[0].axhline(plot_data[f'top_{n}_everything']['og'][target],
                linestyle=markers[target][1],
                linewidth=2,
                color=markers[target][2],
                alpha=alpha)

    if target != 'opt':
        ax[1].errorbar(x=plot_data[f'bot_{n}_everything']['eve']['point'], 
                      y=[np.mean(vals) for vals in plot_data[f'top_{n}_everything']['eve'][target]],
                      yerr=[np.std(vals) for vals in plot_data[f'top_{n}_everything']['eve'][target]],
                      fmt=markers[target][0],
                      color=markers[target][2],
                      markersize=markersize,
                      elinewidth=elinewidth)

        ax[1].axhline(plot_data[f'top_{n}_everything']['og'][target],
                    linestyle=markers[target][1],
                    linewidth=2,
                    color=markers[target][2],
                    alpha=alpha)

ax[0].set_xlabel('N', style='italic')
ax[0].set_ylabel('Overlapped (%)')
ax[1].set_xlabel('N', style='italic')
ax[1].set_ylabel('Overlapped (%)')

ax[0].set_ylim(0, 100)
ax[1].set_ylim(0, 100)
ax[0].set_xlim(0, 8000)
ax[1].set_xlim(0, 8000)

leg1_ele = [Line2D([0], [0], color='dodgerblue', marker='o', markersize=12, linestyle='', label='COF'),
            Line2D([0], [0], color='firebrick', marker='s', markersize=12, linestyle='', label=r'$\mathit{F_0}$'),
            Line2D([0], [0], color='indigo', marker='P', markersize=12, linestyle='', label='Optimal')]
leg2_ele = [Line2D([0], [0], color='dodgerblue', marker='o', markersize=12, linestyle='', label='COF'),
            Line2D([0], [0], color='firebrick', marker='s', markersize=12, linestyle='', label=r'$\mathit{F_0}$')]                         

legend1a = ax[0].legend(handles=leg1_ele, loc=4)#, title='Properties')
ax[0].add_artist(legend1a)

legend1b = ax[1].legend(handles=leg2_ele, loc=4)#, title='Properties')
ax[1].add_artist(legend1b)

#plt.savefig(fname=f'./plots/top_and_bot/overlap_{n}_teve_eve.pdf', dpi=500, bbox_inches='tight')


plt.show()